# IMPORTS

In [38]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# LOAD DATASET

In [54]:
df = pd.read_csv('../datasets/train.csv')

# DATA PREPARATION

In [40]:
features = ['idade',
 'divida_atual',
 'renda_anual',
 'valor_em_investimentos',
 'taxa_utilizacao_credito',
 'num_emprestimos',
 'num_contas_bancarias',
 'num_cartoes_credito',
 'dias_atraso_dt_venc',
 'num_pgtos_atrasados',
 'num_consultas_credito',
 'taxa_juros']
label = ['saldo_atual']

In [41]:
x_train = df.loc[:, features]
y_train = df.loc[:, label]

# MODEL TRAINING

In [42]:
# Model Definition
lr_model = LinearRegression()

# Model Fit
lr_model.fit( x_train, y_train)

# Preview
y_pred = lr_model.predict( x_train )



In [43]:
df1 = df.loc[:, ['id_cliente', 'saldo_atual']]
df1['predicted'] = y_pred
df1.head()

,id_cliente,saldo_atual,predicted
0,1767,278.172008,346.669549
1,11920,268.874152,367.840277
2,8910,446.643127,431.468979
3,4964,321.141267,445.506463
4,10100,428.716114,378.271169


In [44]:
lr_model.intercept_

array([190.82359144])

In [45]:
# Model parameters
lr_model.coef_

array([[ 3.96706202e-04, -4.00595601e-02,  2.77622532e-06,
        -1.04318668e-03,  9.80890872e+00, -1.22353405e-02,
        -6.33015538e-03, -3.57808095e-03, -2.15858165e+00,
         3.77570060e-04,  6.79176336e-03,  3.48471845e-03]])

In [48]:
# y = a + bx
for i in range (len(x_train)):

    print(np.sum((x_train.loc[i,].values * lr_model.coef_)) + lr_model.intercept_)

    if i == 5:
        break

[346.66954862]
[367.84027655]
[431.46897895]
[445.50646329]
[378.27116865]
[352.4833463]


In [50]:
df1['error'] = df1.apply(lambda x: x['saldo_atual'] - x['predicted'], axis=1)

In [51]:
df1.head()

,id_cliente,saldo_atual,predicted,error
0,1767,278.172008,346.669549,-68.497541
1,11920,268.874152,367.840277,-98.966124
2,8910,446.643127,431.468979,15.174148
3,4964,321.141267,445.506463,-124.365197
4,10100,428.716114,378.271169,50.444945
